# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import math

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
df = pd.read_csv('homepage_actions.csv')

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [6]:
click_ids = set(df[df.action == 'click']['id'])
view_ids = set(df[df.action == 'view']['id'])
print("Number of viewers: {} \nNumber of clickers: {}".format(len(view_ids), len(click_ids)))
print("There were {} viewers who didn't click.".format(len(view_ids-click_ids)))
print("There were {} clickers who didn't view.".format(len(click_ids-view_ids)))

Number of viewers: 6328 
Number of clickers: 1860
There were 4468 viewers who didn't click.
There were 0 clickers who didn't view.


In [7]:
# Checked that all clickers are also viewers
len(click_ids&view_ids)

1860

In [8]:
experiment_ids = set(df[df.group == 'experiment']['id'])
control_ids = set(df[df.group == 'control']['id'])
print("Members in experiment group: {}".format(len(experiment_ids)))
print("Members in control group: {}".format(len(control_ids)))
print("Members in both groups: {}".format(len(experiment_ids&control_ids)))

Members in experiment group: 2996
Members in control group: 3332
Members in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [9]:
#H0 = The experimental homepage was equally or less effective than the control homepage at generating clicks.

In [10]:
#H0 = The experimental homepage was more effective than the control homepage at generating clicks.

In [11]:
exp_df = df[df.group == 'experiment'].groupby('action')['id'].count()
control_df = df[df.group == 'control'].groupby('action')['id'].count()

In [12]:
exp_df

action
click     928
view     2996
Name: id, dtype: int64

In [13]:
control_df

action
click     932
view     3332
Name: id, dtype: int64

In [14]:
experimental_clicks = exp_df['click']
experimental_views = exp_df['view']
control_clicks = control_df['click']
control_views = control_df['view']

In [17]:
observed = np.array([experimental_clicks, experimental_views])
expected = np.array([control_clicks, control_views])

In [18]:
stats.chisquare(f_obs=observed, f_exp=expected)

Power_divergenceResult(statistic=33.89952032315072, pvalue=5.8033006040559e-09)

In [19]:
experimental_percent = (experimental_clicks/experimental_views)*100
control_percent = (control_clicks/control_views)*100
print("""Experimental click rate: {}% \nControl click rate: {}%""".format(
         round(experimental_percent, 2), round(control_percent, 2)))

print("""\nDifference in click rate: {}%""".format(round(experimental_percent-control_percent),2))

print("""\nThis difference is significant at the 95% confidence level (alpha = 0.05). It is 
highly unlikely that this difference can be attributed to random chance.
\nTherefore, we can reject the null hypothesis that the experimental homepage was equally or 
less effective than the control homepage at generating clicks.
""")

Experimental click rate: 30.97% 
Control click rate: 27.97%

Difference in click rate: 3.0%

This difference is significant at the 95% confidence level (alpha = 0.05). It is 
highly unlikely that this difference can be attributed to random chance.

Therefore, we can reject the null hypothesis that the experimental homepage was equally or 
less effective than the control homepage at generating clicks.



## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [20]:
# The variance of a binomial distribution depends on n. 
# Therefore, this test isn't applicable: n is different for the two groups.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [21]:
# Solution code says:

# n = len(experiment)     Length of the EXPERIMENTAL group
# p = control_rate        Rate of the CONTROL group
# var = n * p * (1-p)     This is using two different groups: length of experimental but rate of control
# std = np.sqrt(var)

In [30]:
p = control_clicks/control_views #conversion rate for control group
mu = control_clicks #number clicks for control group
n = control_views #number of control views
std = np.sqrt(n1*p1*(1-p1))
print("Control Group")
print("Probability: {} \tMu: {} \tn: {} \tstd: {}".format(p, mu, n, std))

Control Group
Probability: 0.2797118847539016 	Mu: 932 	n: 3332 	std: 25.909622216646923


In [31]:
# If a group of size n had the same rate as the experimental group:

x_bar = (experimental_clicks/experimental_views)*n

print("""Using the click rate of the experimental group, a group of size n (n={}) would have 
{} clicks.""".format(n, round(x_bar)))

Using the click rate of the experimental group, a group of size n (n=3332) would have 
1032.0 clicks.


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [34]:
z = (x_bar - mu)/(std)
z

3.8624556359159157

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [35]:
#Your code here
from scipy.stats import norm

1-stats.norm.cdf(z)

5.6126477469220504e-05

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **While the p-values are different, both are very small and both lead to rejecting the null hypothesis
at the 95% confidence level.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.